<a href="https://colab.research.google.com/github/SushmitaSingh96/WEcomSitePrediction/blob/main/WEcomClothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp -r kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d nicapotato/womens-ecommerce-clothing-reviews

  0% 0.00/2.79M [00:00<?, ?B/s]
100% 2.79M/2.79M [00:00<00:00, 93.9MB/s]


In [ ]:
ls

kaggle.json  sample_data/  womens-ecommerce-clothing-reviews.zip


In [ ]:
!unzip womens-ecommerce-clothing-reviews.zip

Archive:  womens-ecommerce-clothing-reviews.zip
  inflating: Womens Clothing E-Commerce Reviews.csv  


In [ ]:
ll

total 11152
-rw-r--r-- 1 root      72 Jul 19 11:46  kaggle.json
drwxr-xr-x 1 root    4096 Jul 15 13:38  sample_data/
-rw-r--r-- 1 root 8483448 Sep 27  2019 'Womens Clothing E-Commerce Reviews.csv'
-rw-r--r-- 1 root 2924120 Jul 19 11:46  womens-ecommerce-clothing-reviews.zip


In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame(pd.read_csv('Womens Clothing E-Commerce Reviews.csv'))

In [ ]:
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


In [ ]:
df.describe()

,Unnamed: 0,Clothing ID,Age,Rating,Recommended IND,Positive Feedback Count
count,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000
mean,11742.500000,918.118709,43.198544,4.196032,0.822362,2.535936
std,6779.968547,203.298980,12.279544,1.110031,0.382216,5.702202
min,0.000000,0.000000,18.000000,1.000000,0.000000,0.000000
25%,5871.250000,861.000000,34.000000,4.000000,1.000000,0.000000
50%,11742.500000,936.000000,41.000000,5.000000,1.000000,1.000000
75%,17613.750000,1078.000000,52.000000,5.000000,1.000000,3.000000
max,23485.000000,1205.000000,99.000000,5.000000,1.000000,122.000000


In [ ]:
df.drop(['Clothing ID','Age','Division Name','Department Name','Class Name','Title'],axis=1, inplace=True)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
df[df['Review Text'].map(type) != str]['Review Text']

92       NaN
93       NaN
98       NaN
135      NaN
142      NaN
        ... 
23258    NaN
23301    NaN
23303    NaN
23470    NaN
23480    NaN
Name: Review Text, Length: 845, dtype: object

There are 845 rows in column Review Text which are either NaN or some other value which is not a string. So let us convert them to empty strings. 

In [ ]:
df.head()

,Review Text,Rating,Recommended IND,Positive Feedback Count
0,Absolutely wonderful - silky and sexy and comf...,4,1,0
1,Love this dress! it's sooo pretty. i happene...,5,1,4
2,I had such high hopes for this dress and reall...,3,0,0
3,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0
4,This shirt is very flattering to all due to th...,5,1,6


In [ ]:
df['Review Text'].fillna('',inplace=True)

In [ ]:
df[df['Review Text'].map(type) != str]['Review Text']

Series([], Name: Review Text, dtype: object)

Step 1: Remove punctuation and correct spelling. The string library has 32 punctuations. We use the operation string.punctuation to do this task. To correct spelling we use the textblob library.

In [ ]:
df.head()

,Review Text,Rating,Recommended IND,Positive Feedback Count
0,Absolutely wonderful - silky and sexy and comf...,4,1,0
1,Love this dress! it's sooo pretty. i happene...,5,1,4
2,I had such high hopes for this dress and reall...,3,0,0
3,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0
4,This shirt is very flattering to all due to th...,5,1,6


In [ ]:
import string
def removePunc(raw):
  no_punc = [review for review in raw if review not in string.punctuation]
  clean = ''.join(no_punc)
  return clean


df['Review Text'] = df['Review Text'].apply(lambda x: removePunc(x))

In [ ]:
df['Review Text'] = df['Review Text'].astype(str)

In [ ]:
df.head()

,Review Text,Rating,Recommended IND,Positive Feedback Count
0,Absolutely wonderful silky and sexy and comfo...,4,1,0
1,Love this dress its sooo pretty i happened t...,5,1,4
2,I had such high hopes for this dress and reall...,3,0,0
3,I love love love this jumpsuit its fun flirty ...,5,1,0
4,This shirt is very flattering to all due to th...,5,1,6


from textblob import TextBlob

df['Review Text'] = df['Review Text'].apply(lambda x : ''.join(TextBlob(x).correct()))

Step 2: Tokenization: Splitting the string into a list of words. 

In [ ]:
import re
def tokenizer(clean):
  token = re.split('\W',clean)
  return token

df['Review Text'] = df['Review Text'].apply(lambda x: tokenizer(x.lower()))

In [ ]:
df['Review Text'].head()

0    [absolutely, wonderful, , silky, and, sexy, an...
1    [love, this, dress, , its, sooo, pretty, , i, ...
2    [i, had, such, high, hopes, for, this, dress, ...
3    [i, love, love, love, this, jumpsuit, its, fun...
4    [this, shirt, is, very, flattering, to, all, d...
Name: Review Text, dtype: object

Step 3: Stop Words: They are the most common words in any language and do not add much value to the meaning of the document. We do not always need to remove them such when we do language modelling. But in this case we can remove the stop words. They offer benefits like: 
1) dataset size decreases and time to train model as well.
2) increase classification accuracy. 

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopword = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def removeStopwords(text):
  text = [word for word in text if word not in stopword]
  return text

df['Review Text'] = df['Review Text'].apply(lambda x: removeStopwords(x))

In [ ]:
pd.set_option('display.max_colwidth',None)
df['Review Text'].head()

0                                                                                                                                                                                                                                                                                                                   [absolutely, wonderful, , silky, sexy, comfortable]
1                                                                                                                                    [love, dress, , sooo, pretty, , happened, find, store, im, glad, bc, never, would, ordered, online, bc, petite, , bought, petite, 58, , love, length, hits, little, knee, , would, definitely, true, midi, someone, truly, petite]
2    [high, hopes, dress, really, wanted, work, initially, ordered, petite, small, usual, size, found, outrageously, small, small, fact, could, zip, reordered, petite, medium, ok, overall, top, half, comfortable, fit, nicely, bottom, half, tight, layer, several, somewhat, cheap, 

Step 4: Lemmatization: Lemmatization is a more effective option than stemming because it converts the word into its root word, rather than just stripping the suffices. It makes use of the vocabulary and does a morphological analysis to obtain the root word. Therefore, we usually prefer using lemmatization over stemming.

In [ ]:
from textblob import Word

def lemmatizer(text):
  new_words = []
  for word in text:
    new_words.append(Word(word).lemmatize())
    return new_words

  df['Review text'] = df['Review Text'].apply(lambda x: lemmatizer(x))


Step 5: Vectorization

In [ ]:
!pip install gensim

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

cards_doc = [TaggedDocument(doc,[i]) for i,doc in enumerate(df['Review Text'])]


In [ ]:
df.head()

,Review Text,Rating,Recommended IND,Positive Feedback Count
0,"[absolutely, wonderful, , silky, sexy, comfortable]",4,1,0
1,"[love, dress, , sooo, pretty, , happened, find, store, im, glad, bc, never, would, ordered, online, bc, petite, , bought, petite, 58, , love, length, hits, little, knee, , would, definitely, true, midi, someone, truly, petite]",5,1,4
2,"[high, hopes, dress, really, wanted, work, initially, ordered, petite, small, usual, size, found, outrageously, small, small, fact, could, zip, reordered, petite, medium, ok, overall, top, half, comfortable, fit, nicely, bottom, half, tight, layer, several, somewhat, cheap, net, layers, imo, major, design, flaw, net, layer, sewn, directly, zipper, , c]",3,0,0
3,"[love, love, love, jumpsuit, fun, flirty, fabulous, every, time, wear, get, nothing, great, compliments]",5,1,0
4,"[shirt, flattering, due, adjustable, front, tie, perfect, length, wear, leggings, sleeveless, pairs, well, cardigan, love, shirt]",5,1,6


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
def toString(text):
  s = ''
  for word in text:
    s = ''.join(word)
  return s
vect.fit(df["Review Text"].apply(lambda x: toString(x)))
X = vect.transform(df["Review Text"].apply(lambda x: toString(x)))


In [ ]:
y = df["Recommended IND"].copy()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.20, random_state=23, stratify=y)

In [ ]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
print("Train Set Accuracy: {}".format(metrics.accuracy_score(model.predict(X_train), y_train)))
print("Train Set ROC: {}\n".format(metrics.roc_auc_score(model.predict(X_train), y_train)))

print("Validation Set Accuracy: {}".format(metrics.accuracy_score(model.predict(X_valid), y_valid)))
print("Validation Set ROC: {}".format(metrics.roc_auc_score(model.predict(X_valid), y_valid)))

Train Set Accuracy: 0.8452735788801362
Train Set ROC: 0.7872459376865265

Validation Set Accuracy: 0.8386547467007237
Validation Set ROC: 0.7590968030404649


In [ ]:
df.head()

,Review Text,Rating,Recommended IND,Positive Feedback Count
0,"[absolutely, wonderful, , silky, sexy, comfortable]",4,1,0
1,"[love, dress, , sooo, pretty, , happened, find, store, im, glad, bc, never, would, ordered, online, bc, petite, , bought, petite, 58, , love, length, hits, little, knee, , would, definitely, true, midi, someone, truly, petite]",5,1,4
2,"[high, hopes, dress, really, wanted, work, initially, ordered, petite, small, usual, size, found, outrageously, small, small, fact, could, zip, reordered, petite, medium, ok, overall, top, half, comfortable, fit, nicely, bottom, half, tight, layer, several, somewhat, cheap, net, layers, imo, major, design, flaw, net, layer, sewn, directly, zipper, , c]",3,0,0
3,"[love, love, love, jumpsuit, fun, flirty, fabulous, every, time, wear, get, nothing, great, compliments]",5,1,0
4,"[shirt, flattering, due, adjustable, front, tie, perfect, length, wear, leggings, sleeveless, pairs, well, cardigan, love, shirt]",5,1,6


In [ ]:
X

<23486x3481 sparse matrix of type '<class 'numpy.float64'>'
	with 21826 stored elements in Compressed Sparse Row format>